# Import Libraries

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import hashlib
from pinecone import Pinecone
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import json
import ast
from rapidfuzz import fuzz
from datetime import datetime

c:\Codes\Python\notebooks\echo_chatbot\.chatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Initialization

In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("scs")

# OpenAI Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)
LLM = ChatOpenAI(temperature=0, model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)

C:\Users\Zeck Lowe\AppData\Local\Temp\ipykernel_2424\1173229963.py:9: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(api_key=OPENAI_API_KEY)


# Query

In [26]:
query = "Who is the team lead of the project meeting?"

# Generate Embeddings

In [23]:
def get_query_embeddings(query):
    """
    This function returns a list of the embeddings for a given query
    """
    query_embeddings = EMBEDDINGS.embed_query(query)
    print("Generating Embeddings: Done!")
    return query_embeddings

query_embeddings = get_query_embeddings(query=query)
print(query_embeddings)

Generating Embeddings: Done!
[-0.0015675132162868977, 0.02714894525706768, 0.08566092699766159, -0.03447890654206276, -0.0668635368347168, 0.005076061002910137, 0.024760961532592773, 0.006748926360160112, -0.003028078004717827, -0.07120532542467117, 0.03230801224708557, -0.009769023396074772, -0.047785207629203796, -0.03580698370933533, -0.02109598182141781, 0.002622631611302495, -0.03179721161723137, -0.04323910176753998, -0.033508386462926865, -0.0018197200261056423, 0.013319073244929314, 0.035296183079481125, -0.03177167475223541, 0.04939422383904457, -0.04987948387861252, -0.018005650490522385, -0.05792456120252609, 0.035091862082481384, 0.014979168772697449, -0.004376905504614115, -0.012597570195794106, -0.030673455446958542, 0.016958514228463173, 0.005455967504531145, -0.03501524403691292, -0.014442830346524715, -0.002099063014611602, -0.0287324208766222, -0.051539577543735504, -0.004715309478342533, -0.011824987828731537, -0.04313694313168526, -0.02276884764432907, 0.01483869925

# Metadata Filtering

## Fuzzy Match

In [24]:
def fuzzy_match(title1, title2, threshold=80):
    """
    Perform a fuzzy match between two titles using RapidFuzz.
    Returns True if the similarity score is above the threshold.
    """
    similarity_score = fuzz.partial_ratio(title1.lower(), title2.lower())
    return similarity_score >= threshold

## Extract Metadata

In [27]:
def extract_metadata_from_query(query):
  prompt = f"""
  You are a helpful assistant. Extract the meeting title and the meeting date from the following query.
  If the meeting title or date is not explicitly mentioned, return 'unknown'.
  If the date is mentioned as word, it should be formatted as 'YYYY-MM-DD'

  Query: {query}

  Provide the meeting title and date as a Python dictionary in this format:
  {{"meeting_title": "title_here", "date": "date_here"}}
  """

  response = LLM.invoke(prompt)
  metadata_str = response.content.strip()
  metadata_dict = ast.literal_eval(metadata_str)
  return metadata_dict

metadata = extract_metadata_from_query(query)
print(metadata)

{'meeting_title': 'project meeting', 'date': 'unknown'}


# Query Pinecone Index

In [30]:
def query_pinecone_index(query_embeddings, meeting_title, date, top_k=3, include_metadata=True):
    """
    Query a Pinecone index.
    """
    filter_conditions = {}
    if date.lower() != 'unknown':
      filter_conditions['date'] = date
    # if meeting_title.lower() != 'unknown':
    #   filter_conditions['title'] = meeting_title
    # if date.lower() != 'unknown':
    #   filter_conditions['date'] = date

    query_response = index.query(
        vector=query_embeddings,
        filter=filter_conditions,
        top_k=top_k,
        include_metadata=include_metadata,
        namespace="Project Meeting") # Filter based on metadata

    filtered_matches = []
    for match in query_response['matches']:
      if 'metadata' in match and 'title' in match['metadata']:
        metadata_title = match['metadata']['title']
        if fuzzy_match(meeting_title, metadata_title):
          filtered_matches.append(match)

    if not filtered_matches:
      return query_response

    query_response['matches'] = filtered_matches

    print("Querying Pinecone Index: Done!")
    return query_response

answers = query_pinecone_index(query_embeddings=query_embeddings, meeting_title="", date="unknown")
print(answers)

{'matches': [{'id': '0918b01f-57ca-4fd7-94f3-f664a597ddb2',
              'metadata': {'meeting title': 'Project Meeting',
                           'text': 'Czech: Hello my name is Czech.\n'
                                   'Gian: Hello my name is Gian.\n'
                                   'Shaundyl: Hello my name is Shaundyl.\n'
                                   'Czech (Team Lead): Alright, everyone, '
                                   'thanks for joining today’s meeting. We '
                                   'have about 10 minutes to go over the final '
                                   "details before the product launch. Let's "
                                   'start with the progress update. Bob, how '
                                   'are we doing on the development front?\n'
                                   'Shaundyl (Developer): Things are looking '
                                   'good. We’ve implemented all the major '
                                   'fe

# Combining Text from Multiple Document Matches

In [31]:
text_answer = " ".join([doc['metadata']['text'] for doc in answers['matches']])
print(text_answer)

Czech: Hello my name is Czech.
Gian: Hello my name is Gian.
Shaundyl: Hello my name is Shaundyl.
Czech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front?
Shaundyl (Developer): Things are looking good. We’ve implemented all the major features. I’m currently working on the final round of bug fixes. I should be done with it by tomorrow, but I need the QA team to give it another pass afterward. Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the common issues we’re anticipating. We’ll also monitor social media for any unexpected feedback.
Czech: Sounds like we’re in good shape. Thanks, everyone. Let’s aim to regroup tomorrow for a final status check. Anything else before we wrap up?
Shaundyl: Nothing from my side. I’ll update you if any blockers come up.

## Prompt

In [32]:
prompt = f"""You are a meeting facilitator.
        This user will ask you a questions about the conversation of the meeting.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer within 2 sentences and concise.
        Context: {text_answer}
        Question: {query}"""

print(prompt)

You are a meeting facilitator.
        This user will ask you a questions about the conversation of the meeting.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer within 2 sentences and concise.
        Context: Czech: Hello my name is Czech.
Gian: Hello my name is Gian.
Shaundyl: Hello my name is Shaundyl.
Czech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front?
Shaundyl (Developer): Things are looking good. We’ve implemented all the major features. I’m currently working on the final round of bug fixes. I should be done with it by tomorrow, but I need the QA team to give it another pass afterward. Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the co

# LLM

In [33]:
def better_query_response(prompt):
    """
    This function returns a better response using LLM
    """
    better_answer = LLM.invoke(prompt)
    print("Generating Better Response: Done!")
    return better_answer

final_answer = better_query_response(prompt=prompt)
print(final_answer.content)

Generating Better Response: Done!
Czech is the team lead of the project meeting.


# Chatbot Response

In [12]:
def Chatbot(query, meeting_title=None, date=None):
    print(query)
    metadata = extract_metadata_from_query(query)
    print(metadata)
    meeting_title = metadata.get('meeting_title', 'unknown')
    date = metadata.get('date', 'unknown')

    query_embeddings = get_query_embeddings(query=query)

    answers = query_pinecone_index(
        query_embeddings=query_embeddings,
        meeting_title=meeting_title,
        date=date
        )
    print(answers)

    text_answers = " ".join([doc['metadata']['text'] for doc in answers['matches']])
    print(text_answers)
    final_answer = better_query_response(prompt=prompt)
    return final_answer.content

response = Chatbot(query)
print(response)

What are the keypoints from the project meeting?


SyntaxError: invalid syntax (<unknown>, line 1)